In [1]:
import json
import codecs
import csv
import requests
import pandas as pd
from geopy.geocoders import Nominatim
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import os

import time
from geopy.distance import distance
import numpy as np
import io
import random 
from jsonpath_ng import jsonpath, parse

In [2]:
# Nielson coords are in the Albers-USA projection and need to be converted into lat long in order to be used. :/
f = open('nielsen-dma/nielsen-mkt-map.json')
x = json.load(f)
f.close()

In [3]:
lat_expr = parse('features[*].properties[*].latitude')
long_expr = parse('features[*].properties[*].longitude')
dma_expr = parse('features[*].properties[*].dma_name')
coords_expr = parse('features[*].geometry[*].coordinates')

# get lists of individual loc data
lats = [match.value for match in lat_expr.find(x)]
longs = [match.value for match in long_expr.find(x)]
names = [match.value for match in dma_expr.find(x)]
coords = [match.value for match in coords_expr.find(x)]
state = [i[-2:] for i in names]

# get list of lists of the location info to put into a dict with dma name
loc = [[state[i], lats[i], longs[i], coords[i]] for i in range(len(lats))]

dma_dict = dict(zip(names,loc))

In [4]:
print('Min Longitude: ', str(np.min([i for i in longs if i!=None])))
print('Max Longitude: ', str(np.max([i for i in longs if i!=None])))
print('Min Latitude: ', str(np.min([i for i in lats if i!=None])))
print('Min Latitude: ', str(np.max([i for i in lats if i!=None])))

Min Longitude:  -123.9077705
Max Longitude:  -68.751059
Min Latitude:  25.4400565
Min Latitude:  47.8366395


Looks like Alaska gets excluded from lat calcs anyway.

In [5]:
def rad(d): return (d/360.0)*2*np.pi

def USA_Albers(lat, long):
    R = 1.0          # Radius
    phi = lat        # Latitude
    phi_0 = 0      # Reference Latitude
    phi_1 = 25.44      # Std. Parallel 1 (need to fine tune (loss fxn?))
    phi_2 = 47.83     # Std. Parallel 2
    lamda = long      # Longitude
    lamda_0 = 0    # Reference longitude
    
    n = .5*(np.sin(phi_1) + np.sin(phi_2))
    theta = n*(lamda - lamda_0)
    C = np.cos(phi_1)**2 + (2*n*np.sin(phi_1))
    rho = (R/n)*np.sqrt((C-(2*n*sin(phi))))
    rho_0 = (R/n)*np.sqrt((C-(2*n*sin(phi_0))))
    
    lat_new = lat*C         # Not sure what to mult by yet. Above seems built for a projection
    long_new = long*C       # ''
    
    return [lat_new, long_new]

In [6]:
dma_dict['Sacramento-Stockton-Modesto, CA'][1]

38.7901085

In [73]:
# Bad data source. Looking for something more authoritative
zips = pd.read_csv('zip_code_database.csv')
zips['zip'] = zips['zip'].astype(str)
zips['zip_len'] = zips['zip'].apply(lambda x: len(x))
zips = zips[zips.zip_len>=5]

In [77]:
zips['dma'] = 0

In [92]:
geolocator = Nominatim(user_agent="hank_app")


YC_coord = geolocator.geocode('-20.175988, 17.633473').raw
print(YC_coord)


{'place_id': 258510821, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 2279628, 'boundingbox': ['-22.3787758', '-18.7475425', '15.5724597', '20.9990219'], 'lat': '-19.9424962', 'lon': '18.395886', 'display_name': 'Otjozondjupa Region, Namibia', 'class': 'boundary', 'type': 'administrative', 'importance': 0.001, 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//poi_boundary_administrative.p.20.png'}


In [82]:
def point_in_polygon(polygon, point):
    """
    Raycasting Algorithm to find out whether a point is in a given polygon.
    Performs the even-odd-rule Algorithm to find out whether a point is in a given polygon.
    This runs in O(n) where n is the number of edges of the polygon.
     *
    :param polygon: an array representation of the polygon where polygon[i][0] is the x Value of the i-th point and polygon[i][1] is the y Value.
    :param point:   an array representation of the point where point[0] is its x Value and point[1] is its y Value
    :return: whether the point is in the polygon (not on the edge, just turn < into <= and > into >= for that)
    """

    # A point is in a polygon if a line from the point to infinity crosses the polygon an odd number of times
    odd = False
    # For each edge (In this case for each point of the polygon and the previous one)
    i = 0
    j = len(polygon) - 1
    while i < len(polygon) - 1:
        i = i + 1
        # If a line from the point into infinity crosses this edge
        # One point needs to be above, one below our y coordinate
        # ...and the edge doesn't cross our Y corrdinate before our x coordinate (but between our x coordinate and infinity)

        if (((polygon[i][1] > point[1]) != (polygon[j][1] > point[1])) and (point[0] < (
                (polygon[j][0] - polygon[i][0]) * (point[1] - polygon[i][1]) / (polygon[j][1] - polygon[i][1])) +
                                                                            polygon[i][0])):
            # Invert odd
            odd = not odd
        j = i
    # If the number of crossings was odd, the point is in the polygon
    return odd


In [90]:
# Need to get coords out of Albers-USA
point_in_polygon(dma_dict['Sacramento-Stockton-Modesto, CA'][2][0][0], (38.575764, -121.478851))

False